In [1]:
# Resolve path when used in a usecase project
import sys
from pathlib import Path

sys.path.insert(0, str(Path("../../").resolve()))

# Tutorial about creating reports

This tutorial makes the user familiar with the process or creating a report creations and provide the basic knowledge for creating custom reports.

## TL;DR
- Create a report structure by putting the content in a dictionary (the keys will become the headers)
- Render it with `generate_html_report`
- This code makes a report with two sections with one figure each
    
    ```python
    from reporting.rendering.html import generate_html_report

    report_structure = {
        "A Nice Title": figure1,
        "Another Nice Title": figure2,
    }
    generate_html_report(
        report_structure=report_structure,
        render_path="path/to/report.html",
    )
    ```

In [2]:
import plotly.graph_objects as go
from reporting.rendering.html import generate_html_report
from reporting.datasets import create_basic_report_structure

# Save an html report
generate_html_report(
    report_structure=create_basic_report_structure(),
    render_path="./report_examples/tldr.html",
)

In [3]:
!open ./report_examples/tldr.html

## Creating reports using `reporting.reports`

The `reporting.reports` module allows to assembling charts into organic reports describing a common topic.  

This module
- Provides functionality for assembling custom reports.  
   These reports can be composed choosing the most suitable content to be helpful during team PS, client interaction etc..
- Additionally provides four out-of-the-box reports (see `plot_batches_profiles`, `plot_sensor_trends`, `plot_feature_overviews`, `get_modeling_overview` for more details)


### Report composition is distinct from report generation

In the `reporting` pacakge, "report" refers to the concept of a nested structure of sections, comprising of header and content.  
These sections can be nested, so the content of a section can be another (sub)section, which in turns has its (sub)header and content.

The key concept is the distinction between
- the report structure described above, which represents how the information is organized
- the "physical" rendering of the report, which is an html file (or a pdf in the future - interactive is a bonus at the moment) built according to the report structure.

This tutorial makes the user familiar with report structures, and shows how to convert them into a html files (in the future pdf and interactive).


### A report is created by first assembling the content and then rendering it

#### The report structure

As described above, the report structure contains the information on how the content of a report is organized in sections and subsections.

In the code, this structure is represented by a nested dictionary, where the keys are the headers of the sections (and of all the subsections), and the values are the content.  
The content can be
- any python object of the supported types (in short: figures, dataframes, and most of the basic object needed in practice are supported. See .... in `api.types` for details)
- a list of these objects, in which case they will be rendered one after each other within the same section
- a (nested) dictionary, which will then be used to create subheaders and subsections within a section

Once a structure is created, then the report can be generated by using one of the renderers described in the next section.


#### Rendering the report
A report structure is then passed as input to a report generator, which creates the rendered report.
These report generators are:
- `generate_html_report` - creates an html file
- (not available yet) `generate_interactive_report` - creates an interactive report to explore the content interactively on jupyter notebooks
- (not available yet) `generate_pdf_report` - creates a pdf file

In this tutorial reports will be saved as html files using `generate_html_report`.

## Practical examples

### 1. Creating report with a simple structure
The simplest report structure is the one where every section contains one figure.

The report strucuture is a dictionary containing the figures, like this one.

```python
# two sections with one figure each.
report_structure = {
    "A Nice Title": figure1,
    "Another Nice Title": figure2,
}
generate_html_report(
    report_structure=report_structure,
    render_path="path/to/report.html",
)
```

Try a practical example in the TL;DR section above.



### 2. Creating report with multiple objects in the same section

Create a report with two figures in the same section.


This example is one step more advanced than the one in the TL;DR, since it uses a list to indicate that a section contains multiple objects.

The code below creates a structure similar to this one.  
When the report is rendered the figures in the list are under the same section.

```python
# two sections; first with one plot, second with two
report_structure = {
    "A section with one element": figure1,
    "A section with two elements": [figure2, figure3],
}
```



In [4]:
import copy
import plotly.graph_objects as go
from reporting.rendering.html import generate_html_report
from reporting.datasets import create_advanced_report_structure

# Save an html report
generate_html_report(
    report_structure=create_advanced_report_structure(),
    render_path="./report_examples/flat_structure.html",
)

In [5]:
!open ./report_examples/flat_structure.html

### 3. Creating a report with a multilevel structure


Create a report with sections and subsections.


This example shows how to create report structures to generate reports with several levels of headers and subheaders.

The code below creates a structure similar to this one.  
When the report is rendered, the sections are nested inside each other in the same way as the dictionaries are nested within the report structure.

```python
# model performance section has two subsections
report_structure = {
    "A simple section": figure1,
    "A section with subsections": {
          "A subsection": figure2,
          "Another subsection": [figure3, figure4],
   }
}

```


In [6]:
import copy
import plotly.graph_objects as go
from reporting.rendering.html import generate_html_report
from reporting.datasets import create_multilevel_report_structure

# Save an html report
generate_html_report(
    report_structure=create_multilevel_report_structure(),
    render_path="./report_examples/multilevel_structure.html",
)

In [7]:
!open ./report_examples/multilevel_structure.html

### 3. Leveraging an out-of-the-box report structure

Create a report using one of the out-of the box structures.
It works in the same way, just create the report structure by using one of the provided out-of-the-box functions.

Use the `plot_feature_overviews` function to generate a report structure containing an out-of-the-box feature overview.  
This functions takes a `pd.Dataframe` as input and creates a dict with overview plots for the desired features.

Leverage the mock data from `get_mill_data()` to have a realistic-looking overview.

Notice that the function returns a dictionary like the one below, where the values are plotly figures.
```python
feature_overviews_report_structure = {
  "mill_a_power": <figure>,
  "mill_a_load": <figure>,
}
```
Render the report structure by using `generate_html_report`, just like with all the other report structures above

In [8]:
from reporting.datasets import get_mill_data
from reporting.reports import plot_feature_overviews

# Use `plot_feature_overviews` to produce a feature overview report structure based on mock data
feature_overviews_report_structure = plot_feature_overviews(
    data=get_mill_data(),
    features=[
        "mill_a_power",
        "mill_a_load",
    ],
    timestamp="time_col",
)

# The report structure created by `plot_feature_overviews` is a simple dictionary
# - the keys are feature names
# - the values are plotly figures with info about each feature
feature_overviews_report_structure.keys()

dict_keys(['mill_a_power', 'mill_a_load'])

In [9]:
# Generate the html report as usual
generate_html_report(
    report_structure=feature_overviews_report_structure,
    render_path="./report_examples/out_of_the_box_feature_overview.html",
)

In [10]:
!open ./report_examples/out_of_the_box_feature_overview.html

## Explore different types of content

All the renderers support the following types of content
- plotly figure
- matplotlib figure
- pandas dataframe

The exact list of explicitly supported content can be found in `api.types`.
Other types may be supported by specific renderers only. If needed, check the details about the specific renderer.

Generate a report structure with instances of these types and see how it works.


In [11]:
%%capture
#Note: the %% capture above just prevents the cell from showing matplotlib output

import plotly.graph_objects as go
import matplotlib.pyplot as plt
import pandas as pd

x = [0, 1, 2, 3, 4, 5]
y = [0, 1, 2, 4, 7, 9]

x_label = "Usage of the reporting package"
y_label = "Time saved by using the reporting package"
main_title = "The reporting package is awesome"

# Create objects of the supported types

matplotlib_figure, ax = plt.subplots()
ax.plot(x, y)
ax.set_xlabel(x_label)
ax.set_ylabel(y_label)
ax.set_title(main_title)

plotly_figure = (
    go.Figure()
    .add_trace(go.Scatter(x=x, y=y))
    .update_xaxes(title=x_label)
    .update_yaxes(title=y_label)
    .update_layout(title=main_title)
)

pandas_dataframe = pd.DataFrame({x_label: x, y_label: y})

In [12]:


report_structure = {
    "Figures": {
        "Matplotlib figure": matplotlib_figure,
        "Plotly figure": plotly_figure,
    },
    "Pandas dataframe": pandas_dataframe,
}

# Generate the html report as usual
generate_html_report(
    report_structure=report_structure,
    render_path="./report_examples/supported_types.html",
)

In [13]:
!open ./report_examples/supported_types.html

## Further reading
For more advanced usage and more customization options, refer to the specific how-to guides in the `reporting` package.